Setup

In [ ]:
# this step reloads machine in order to prevent any error
%load_ext autoreload
%autoreload 2

In [10]:
# import statements
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
import time

import numpy as np
import pandas as pd

In [2]:
# defines the jubilee to call
jubilee = Jub.Machine(address='192.168.1.2', simulated = False) 

0


KeyboardInterrupt: 

In [ ]:
# load deck
deck = jubilee.load_deck('lab_automation_deck.json')

In [ ]:
# this loads the well plate for the destination for samples (sample_holder_name, position)
samples = jubilee.load_labware('fisherbrand_96_wellplate_360ul.json', 2)
# this line of code defines the boundary conditions of the well plate requires 3 points
upper_left = (25.7,179.9)
upper_right = (124.9, 179.9)
lower_right = (124.9, 116.4)
samples.manual_offset([upper_left,upper_right,lower_right])
samples.load_manualOffset()

In [ ]:
# this is same as lines above just for stock solution
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 3)
upper_left = (167.4,176.8)
upper_right = (258.0, 176.8)
lower_right = (258.0, 119.8)
stocks.manual_offset([upper_left,upper_right,lower_right])
stocks.load_manualOffset()

In [ ]:
# same as above just for pipette tip rack
tiprack = jubilee.load_labware('opentrons_96_tiprack_300ul.json', 0)
upper_left = (27.0,82.7)
upper_right = (125.6,82.7)
lower_right = (125.8,19.4)
tiprack.manual_offset([upper_left,upper_right,lower_right])
tiprack.load_manualOffset()

In [ ]:
# defines position of trash can for tips
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

In [ ]:
deck.safe_z

In [ ]:
# this code block defines all of our stocks
water_stock = stocks[0].bottom(+7)
red_stock = stocks[2].bottom(+7)
blue_stock = stocks[1].bottom(+7)
# include however many we need, remember to include however many 

Load Tools

In [ ]:
P300 = Pipette.Pipette.from_config(3,'Pipette','P300_config.json')
jubilee.load_tool(P300)

P300.add_tiprack(tiprack)
P300.trash = trash[0]

Experiment Funcitons

In [5]:
def transfer_stock(v,stock, sample):
    '''
    This function will take a volume to add, and from which stock to use

    Input - v (volume)(float)
          - stock (int) -position of the desired stock
          - sample (int) -position of the current sample
    Output - void, purpose of function    
    '''
    P300.pick_up_tip()
    P300.transfer(vol=v, source_well=stocks[stock].bottom(+7) ,destination_well= samples[sample].bottom(+7))
    P300.return_tip()
    return none

In [3]:
#def transfer_blue(v, sample):
    '''
    This function will take a volume to add, and from which stock to use

    Input - v (volume)(float)
          - stock (int)

    Output - void, purpose of function    
    '''

    #P300.transfer(vol=v, source_well=stocks[2] ,destination_well=samples[sample] , blowout=none, new_tip='once')
    #return none


IndentationError: unexpected indent (206793878.py, line 2)

In [8]:
def transfer_water(v,sample):
    '''
    This function will take a volume to add, and from which stock to use

    Input - v (volume)(float)
          - sample (int)
    Output - void, purpose of function    
    '''
    # source well is always the same
    P300.pick_up_tip()
    P300.transfer( vol=v, source_well=stocks[0].bottom(+5) ,destination_well = samples[sample].bottom(+5) )
    P300.return_tip()
    return none

In [6]:
def mixer(sample_location):
    '''
    this function will mix a chosen sample
    Input - sample_location(int) is an index location of where to mix
    '''
    P300.pick_up_tip()
    P300.mix(500,3,location=samples[sample_location])
    P300.return_tip()
    return none

Data for experiment

In [11]:
data = pd.read_csv('./data/for_igi.csv')

In [12]:
# this is first time step, all the others can be done by increasing the index
# the dt1 is a list of [vol_a, vol_b, vol_h20, stock_a, stock_b]
#dt1 = data.loc[0]
data.head()

,Unnamed: 0,step,CA_target,CB_target,vol_A,vol_B,vol_water,Current_CA,Current_CB,Current_V
0,0,1,5.0,0.00,1000.0,0.0,0.0,5.00,0.00,1000
1,1,2,4.0,0.10,600.0,200.0,200.0,4.00,0.10,2000
2,2,3,3.0,0.20,200.0,400.0,400.0,3.00,0.20,3000
3,3,4,2.0,0.20,0.0,0.0,1000.0,2.25,0.15,4000
4,4,5,2.0,0.25,200.0,650.0,150.0,2.00,0.25,5000


In [20]:
vol_a = data['vol_A']
vol_b = data['vol_B']
vol_water = data['vol_water']

In [ ]:
jubilee.pickup_tool(P300)

In [28]:
# for sample 1
#print(dt1)
#print(dt1.iloc[2])
#vol_a = [1000,600,200]
#vol_b = [0,200,400]
#vol_water= [0,200,400]

In [ ]:
'''# for sample 1 dt1
transfer_red(vol_a[0], sample[0])
transfer_blue(vol_b[0], sample[0])
transfer_water(vol_water[0],sample[0])
mixer(0)
'''

In [ ]:
'''# want to see if it will work for a loop
for i in range(len(vol_a)):
    transfer_red(vol_a[i], sample[0])
    transfer_blue(vol_b[i], sample[0])
    transfer_water(vol_water[i],sample[0])
    #mixer(sample[0])
'''

In [32]:
# this one is for stock a
transfer_stock(vol_a[0],0,0)

NameError: name 'P300' is not defined

In [ ]:
# stock b
transfer_stock(vol_b[0],1,0)

In [ ]:
# water
transfer_water(vol_b[0],0)

In [ ]:
# mix test
mixer(0)